In [2]:
!pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/11/c5/2370d96b31eb1841c3a0883a492c15278a6718ccad61bb6a649c80d1d9eb/torch-2.6.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for sympy==1.13.1 from https://files.pythonhosted.org/packages/b2/fe/81695a1aa331a842b582453b605175f419fe8540355886031328089d840a/sympy-1.13.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/204.2 MB 2.8 MB/s eta 0:01:13
   ---------------------------------------- 0.7/204.2 MB 7.7 MB/s eta 0:00:27
   ---------------------------------------- 1.0/204.2 MB 8.9 MB/s eta 0:00:23
   ---------------------------------------- 1.8/204.2 MB 9.5 MB/s eta 0:00:22
   ---------------------------------------- 2.3/204.2 MB 9.9 MB/s eta 0:00:21
    --------------------------------------- 2.7/204.2 MB 10.9 MB/s eta 0:00:19
    --------------------------------------- 3.6/204.

In [6]:
!pip install torch torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/88/53/4ad334b9b1d8dd99836869fec139cb74a27781298360b91b9506c53f1d10/torchvision-0.21.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
    --------------------------------------- 0.0/1.6 MB 435.7 kB/s eta 0:00:04
   ---- ----------------------------------- 0.2/1.6 MB 1.5 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.6 MB 3.4 MB/s eta 0:00:01
   ----------------------- ---------------- 0.9/1.6 MB 4.4 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.6 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 5.2 MB/s eta 0:00:00


In [8]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from models import Generator, Discriminator  # Assuming we define CycleGAN models in models.py

ModuleNotFoundError: No module named 'models'

In [ ]:
# Hyperparameters
batch_size = 8
epochs = 50
learning_rate = 0.0002
image_size = 256


In [ ]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [ ]:
# Load Datasets (Plain T-shirts & T-shirts with logos)
plain_tshirt_dataset = datasets.ImageFolder("data/plain_tshirts", transform=transform)
logo_tshirt_dataset = datasets.ImageFolder("data/logo_tshirts", transform=transform)

plain_loader = DataLoader(plain_tshirt_dataset, batch_size=batch_size, shuffle=True)
logo_loader = DataLoader(logo_tshirt_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Initialize Models
generator = Generator()
discriminator = Discriminator()
generator.apply(weights_init)
discriminator.apply(weights_init)


In [ ]:
# Loss & Optimizer
criterion = nn.MSELoss()
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))


In [ ]:
# Training Loop
for epoch in range(epochs):
    for (plain_imgs, _), (logo_imgs, _) in zip(plain_loader, logo_loader):
        plain_imgs, logo_imgs = plain_imgs.cuda(), logo_imgs.cuda()

        # Generate Mockup (Fake Logo T-shirt)
        fake_logo_imgs = generator(plain_imgs)

        # Train Discriminator
        real_loss = criterion(discriminator(logo_imgs), torch.ones_like(logo_imgs))
        fake_loss = criterion(discriminator(fake_logo_imgs.detach()), torch.zeros_like(fake_logo_imgs))
        d_loss = (real_loss + fake_loss) / 2

        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        # Train Generator
        g_loss = criterion(discriminator(fake_logo_imgs), torch.ones_like(fake_logo_imgs))
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")


In [ ]:
# Save Model
torch.save(generator.state_dict(), "mockup_generator.pth")
